In [1]:
import os
os.environ["KMP_DUPLICATE_LIB_OK"]="TRUE"

## 层和块

In [2]:
import torch
from torch import nn
from torch.nn import functional as F

net = nn.Sequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))

X = torch.rand(2, 20)
net(X)

tensor([[-0.0102, -0.1001,  0.0691,  0.0867,  0.1132,  0.1750, -0.2894, -0.0751,
         -0.0740, -0.0715],
        [-0.2994, -0.1403, -0.1873,  0.1786,  0.0187,  0.1005, -0.3203, -0.0048,
          0.0556, -0.1726]], grad_fn=<AddmmBackward>)

In [3]:
class MLP(nn.Module):
    # 用模型参数声明层。这里，我们声明两个全连接的层
    def __init__(self):
        # 调用`MLP`的父类`Block`的构造函数来执行必要的初始化。
        # 这样，在类实例化时也可以指定其他函数参数，例如模型参数`params`（稍后将介绍）
        super().__init__()
        self.hidden = nn.Linear(20, 256)  # 隐藏层
        self.out = nn.Linear(256, 10)  # 输出层

    # 定义模型的正向传播，即如何根据输入`X`返回所需的模型输出
    def forward(self, X):
        # 注意，这里我们使用ReLU的函数版本，其在nn.functional模块中定义。
        return self.out(F.relu(self.hidden(X)))

net = MLP()
net(X)

tensor([[-0.1272, -0.0127, -0.2614,  0.2203,  0.1353, -0.0219, -0.0113,  0.3042,
          0.2678,  0.1551],
        [-0.1075,  0.0215, -0.1613,  0.2034,  0.1239,  0.0526,  0.0083,  0.3234,
          0.1507,  0.0435]], grad_fn=<AddmmBackward>)

In [4]:
#自己实现一个nn.Sequential  
class MySequential(nn.Module):
    def __init__(self, *args):
        super().__init__()
        for block in args:
            # 这里，`block`是`Module`子类的一个实例。我们把它保存在'Module'类的成员变量
            # `_modules` 中。`block`的类型是OrderedDict（按序字典）。
            self._modules[block] = block

    def forward(self, X):
        # OrderedDict保证了按照成员添加的顺序遍历它们
        for block in self._modules.values():
            X = block(X)
        return X

net = MySequential(nn.Linear(20, 256), nn.ReLU(), nn.Linear(256, 10))
net(X)

tensor([[ 0.1596,  0.1052, -0.1324,  0.0686, -0.3469,  0.0234,  0.0650,  0.2119,
         -0.0950,  0.0637],
        [-0.0105,  0.0974, -0.1529,  0.0414, -0.2242, -0.0234,  0.1114,  0.1215,
         -0.1031, -0.0023]], grad_fn=<AddmmBackward>)

In [5]:
#可以自定义前向计算
class FixedHiddenMLP(nn.Module):
    def __init__(self):
        super().__init__()
        # 不计算梯度的随机权重参数。因此其在训练期间保持不变。
        self.rand_weight = torch.rand((20, 20), requires_grad=False)
        self.linear = nn.Linear(20, 20)

    def forward(self, X):
        X = self.linear(X)
        # 使用创建的常量参数以及`relu`和`dot`函数。
        X = F.relu(torch.mm(X, self.rand_weight) + 1)
        # 复用全连接层。这相当于两个全连接层共享参数。
        X = self.linear(X)
        # 控制流
        while X.abs().sum() > 1:
            X /= 2
        return X.sum()

net = FixedHiddenMLP()
net(X)

tensor(-0.2148, grad_fn=<SumBackward0>)

In [6]:
#混合搭配各种组合块的方法
class NestMLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.net = nn.Sequential(nn.Linear(20, 64), nn.ReLU(),
                                 nn.Linear(64, 32), nn.ReLU())
        self.linear = nn.Linear(32, 16)

    def forward(self, X):
        return self.linear(self.net(X))

chimera = nn.Sequential(NestMLP(), nn.Linear(16, 20), FixedHiddenMLP())
chimera(X)

tensor(-0.2288, grad_fn=<SumBackward0>)

## 参数管理

In [7]:
import torch
from torch import nn

net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(), nn.Linear(8, 1))
X = torch.rand(size=(2, 4))
net(X)

tensor([[-0.2265],
        [-0.2268]], grad_fn=<AddmmBackward>)

In [8]:
print(net[2].state_dict()) 

OrderedDict([('weight', tensor([[ 0.1923,  0.1604, -0.2987,  0.2388, -0.1111, -0.0150, -0.1151, -0.1109]])), ('bias', tensor([-0.3374]))])


In [9]:
#可以拿具体的参数 
print(type(net[2].bias))
print(net[2].bias)
print(net[2].bias.data)

<class 'torch.nn.parameter.Parameter'>
Parameter containing:
tensor([-0.3374], requires_grad=True)
tensor([-0.3374])


In [10]:
#可以通过 .grad 来访问梯度 ，这里没有进行反向计算所以暂时没有梯度
net[2].weight.grad == None

True

In [11]:
#拿出所有参数
#relu没有参数所以拿不出来
print(*[(name, param.shape) for name, param in net[0].named_parameters()])
print(*[(name, param.shape) for name, param in net.named_parameters()])

('weight', torch.Size([8, 4])) ('bias', torch.Size([8]))
('0.weight', torch.Size([8, 4])) ('0.bias', torch.Size([8])) ('2.weight', torch.Size([1, 8])) ('2.bias', torch.Size([1]))


In [12]:
net.state_dict()['2.bias'].data

tensor([-0.3374])

In [13]:
def block1():
    return nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                         nn.Linear(8, 4), nn.ReLU())

def block2():
    net = nn.Sequential()
    for i in range(4):
        # 在这里嵌套四个block1 ，但是下面这样的写法可以给定具体的名字 ，而不是0，1，2，3。。。
        net.add_module(f'block {i}', block1())
    return net

rgnet = nn.Sequential(block2(), nn.Linear(4, 1))
rgnet(X)

tensor([[0.0816],
        [0.0816]], grad_fn=<AddmmBackward>)

In [14]:
print(rgnet)

Sequential(
  (0): Sequential(
    (block 0): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 1): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 2): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
    (block 3): Sequential(
      (0): Linear(in_features=4, out_features=8, bias=True)
      (1): ReLU()
      (2): Linear(in_features=8, out_features=4, bias=True)
      (3): ReLU()
    )
  )
  (1): Linear(in_features=4, out_features=1, bias=True)
)


In [15]:
rgnet[0][1][0].bias.data

tensor([ 0.0490, -0.1798,  0.1002,  0.1149,  0.4065, -0.1793, -0.3597,  0.4107])

-内置初始化

In [16]:
def init_normal(m):
    if type(m) == nn.Linear:
        nn.init.normal_(m.weight, mean=0, std=0.01) #均值为0 ，方差为0.01
        nn.init.zeros_(m.bias)

#对net里面所有的module当作参数传入init_normal
net.apply(init_normal)
net[0].weight.data[0],net[0].bias.data[0]

(tensor([ 0.0006, -0.0211, -0.0015,  0.0054]), tensor(0.))

In [17]:
def init_constant(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 1)
        nn.init.zeros_(m.bias)
net.apply(init_constant)
net[0].weight.data[0], net[0].bias.data[0]

(tensor([1., 1., 1., 1.]), tensor(0.))

In [18]:
def xavier(m):
    if type(m) == nn.Linear:
        nn.init.xavier_uniform_(m.weight)
def init_42(m):
    if type(m) == nn.Linear:
        nn.init.constant_(m.weight, 42)

net[0].apply(xavier)
net[2].apply(init_42)
print(net[0].weight.data[0])
print(net[2].weight.data)

tensor([-0.5695, -0.3619, -0.5075,  0.1719])
tensor([[42., 42., 42., 42., 42., 42., 42., 42.]])


In [19]:
def my_init(m):
    if type(m) == nn.Linear:
        print("Init", *[(name, param.shape)
                        for name, param in m.named_parameters()][0])
        nn.init.uniform_(m.weight, -10, 10)
        m.weight.data *= m.weight.data.abs() >= 5 #保留绝对值大于5的权重

net.apply(my_init)
net[0].weight[:2]

Init weight torch.Size([8, 4])
Init weight torch.Size([1, 8])


tensor([[ 0.0000, -5.8757, -7.6458,  7.0907],
        [-0.0000, -0.0000,  0.0000,  0.0000]], grad_fn=<SliceBackward>)

In [20]:
net[0].weight.data[:] += 1
net[0].weight.data[0, 0] = 42
net[0].weight.data[0]

tensor([42.0000, -4.8757, -6.6458,  8.0907])

In [21]:
#参数绑定
# 我们需要给共享层一个名称，以便可以引用它的参数。
shared = nn.Linear(8, 8)
#第二个和第三个隐藏层权重永远是一样的
net = nn.Sequential(nn.Linear(4, 8), nn.ReLU(),
                    shared, nn.ReLU(),
                    shared, nn.ReLU(),
                    nn.Linear(8, 1))
net(X)
# 检查参数是否相同
print(net[2].weight.data[0] == net[4].weight.data[0])
net[2].weight.data[0, 0] = 100
# 确保它们实际上是同一个对象，而不只是有相同的值。
print(net[2].weight.data[0] == net[4].weight.data[0])

tensor([True, True, True, True, True, True, True, True])
tensor([True, True, True, True, True, True, True, True])


## 自定义层

自定义一个没有任何参数的层

In [22]:
import torch
import torch.nn.functional as F
from torch import nn


class CenteredLayer(nn.Module):
    def __init__(self):
        super().__init__()

    def forward(self, X):
        return X - X.mean()

layer = CenteredLayer()
layer(torch.FloatTensor([1, 2, 3, 4, 5]))

tensor([-2., -1.,  0.,  1.,  2.])

In [23]:
net = nn.Sequential(nn.Linear(8, 128), CenteredLayer())

In [24]:
Y = net(torch.rand(4, 8))
Y.mean()

tensor(-1.8626e-09, grad_fn=<MeanBackward0>)

定义一个带有参数的层

In [25]:
#这里自己实现一个linear层
class MyLinear(nn.Module):
    def __init__(self, in_units, units):
        super().__init__()
        self.weight = nn.Parameter(torch.randn(in_units, units))
        self.bias = nn.Parameter(torch.randn(units,))
    def forward(self, X):
        linear = torch.matmul(X, self.weight.data) + self.bias.data
        return F.relu(linear)

linear = MyLinear(5, 3)
linear.weight

Parameter containing:
tensor([[ 0.8022,  0.9697, -1.0244],
        [-0.0861,  1.1356,  0.4368],
        [-0.3920,  0.8581,  0.1540],
        [-0.8247, -0.7684, -0.4534],
        [-0.1944, -0.5384, -0.7694]], requires_grad=True)

In [26]:
linear(torch.rand(2, 5))

tensor([[1.4249, 0.6285, 0.0000],
        [1.8507, 1.0183, 0.0000]])

In [27]:
net = nn.Sequential(MyLinear(64, 8), MyLinear(8, 1))
net(torch.rand(2, 64))

tensor([[0.],
        [0.]])

## 文件读写 

加载和保存张量

In [30]:
import torch
from torch import nn
from torch.nn import functional as F

x = torch.arange(4)
torch.save(x, 'x-file')
x2 = torch.load('x-file')
x2

tensor([0, 1, 2, 3])

In [31]:
y = torch.zeros(4)
torch.save([x, y],'x-files')
x2, y2 = torch.load('x-files')
(x2, y2)

(tensor([0, 1, 2, 3]), tensor([0., 0., 0., 0.]))

In [32]:
#字典的形式
mydict = {'x': x, 'y': y}
torch.save(mydict, 'mydict')
mydict2 = torch.load('mydict')
mydict2

{'x': tensor([0, 1, 2, 3]), 'y': tensor([0., 0., 0., 0.])}

加载和保存模型参数

In [33]:
class MLP(nn.Module):
    def __init__(self):
        super().__init__()
        self.hidden = nn.Linear(20, 256)
        self.output = nn.Linear(256, 10)

    def forward(self, x):
        return self.output(F.relu(self.hidden(x)))

net = MLP()
X = torch.randn(size=(2, 20))
Y = net(X)

In [42]:
#把参数存成字典
torch.save(net.state_dict(), 'mlp.params')
#展示部分数据
net.state_dict()['hidden.weight']

tensor([[ 0.0895, -0.1003, -0.1240,  ...,  0.1486,  0.1218, -0.1184],
        [ 0.0650, -0.0080,  0.1079,  ...,  0.0100, -0.1780, -0.0531],
        [ 0.0836,  0.1195,  0.1820,  ...,  0.1806,  0.2188, -0.1974],
        ...,
        [ 0.1723,  0.0892, -0.0595,  ..., -0.0588, -0.1974, -0.0082],
        [ 0.2127,  0.1988,  0.0857,  ..., -0.1862, -0.1906,  0.0385],
        [ 0.1593,  0.0399, -0.0742,  ...,  0.2129, -0.0237, -0.0647]])

In [43]:
#实例化原始多层感知机的备份。直接读取文件中存储的参数
clone = MLP()
clone.load_state_dict(torch.load('mlp.params'))
clone.eval()

MLP(
  (hidden): Linear(in_features=20, out_features=256, bias=True)
  (output): Linear(in_features=256, out_features=10, bias=True)
)

In [44]:
#校验是否一样
Y_clone = clone(X)
Y_clone == Y

tensor([[True, True, True, True, True, True, True, True, True, True],
        [True, True, True, True, True, True, True, True, True, True]])